In [1]:
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [2]:

# Cargar los datos del conjunto de datos CSV
df = pd.read_csv("dataset.csv")

# Convertir los valores de píxeles de cadena de texto a matrices de numpy
pixel_values = df["pixels"].apply(lambda x: np.fromstring(x, dtype=int, sep=" "))
X = np.vstack(pixel_values.values)
X = X.reshape(-1, 48, 48, 1)

# Convertir las etiquetas de las emociones a valores categóricos
y = to_categorical(df["emotion"], num_classes=5)

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
training_samples = int(len(X) * 0.5)
validation_samples = int(len(X) * 0.25)
X_train = X[:training_samples]
y_train = y[:training_samples]
X_val = X[training_samples:training_samples+validation_samples]
y_val = y[training_samples:training_samples+validation_samples]
X_test = X[training_samples+validation_samples:]
y_test = y[training_samples+validation_samples:]


In [3]:
df = pd.read_csv("dataset.csv")
df

,emotion,pixels,Usage
0,0,169 171 163 67 59 74 72 96 145 160 166 174 183...,PublicTest
1,0,178 174 170 104 71 71 77 75 138 156 172 179 18...,Training
2,0,180 178 168 82 69 71 68 100 158 162 174 183 18...,PublicTest
3,0,182 175 175 89 71 72 72 88 152 161 170 177 180...,Training
4,0,180 176 179 78 73 76 77 100 157 167 174 177 18...,PublicTest
...,...,...,...
984,3,178 181 176 171 183 186 178 118 70 73 70 67 10...,PrivateTest
985,3,180 179 181 179 178 180 182 178 130 75 69 66 6...,Training
986,3,176 179 182 177 184 186 176 167 82 65 74 72 76...,Training
987,3,179 180 175 183 182 180 187 171 129 71 65 68 6...,PrivateTest


In [4]:

# Definir el modelo de la red neuronal convolucional
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(5, activation="softmax"))

# Compilar el modelo
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])



In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3965056   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [6]:
# Entrenar el modelo
model.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
16/16 [==============================] - 21s 235ms/step - loss: 122.5404 - accuracy: 0.3148 - val_loss: 4.4413 - val_accuracy: 0.3482
Epoch 2/30
16/16 [==============================] - 3s 198ms/step - loss: 0.7626 - accuracy: 0.7398 - val_loss: 5.2424 - val_accuracy: 0.0000e+00
Epoch 3/30
16/16 [==============================] - 3s 198ms/step - loss: 0.2909 - accuracy: 0.9016 - val_loss: 6.2549 - val_accuracy: 0.2632
Epoch 4/30
16/16 [==============================] - 3s 196ms/step - loss: 0.1095 - accuracy: 0.9531 - val_loss: 8.7650 - val_accuracy: 0.4049
Epoch 5/30
16/16 [==============================] - 3s 181ms/step - loss: 0.0866 - accuracy: 0.9602 - val_loss: 8.4173 - val_accuracy: 0.2470
Epoch 6/30
16/16 [==============================] - 3s 176ms/step - loss: 0.0464 - accuracy: 0.9762 - val_loss: 9.4009 - val_accuracy: 0.3806
Epoch 7/30
16/16 [==============================] - 3s 187ms/step - loss: 0.0196 - accuracy: 0.9928 - val_loss: 10.7330 - val_accuracy: 0.417

In [7]:
#guardamos el modelo
model.save('diego.h5')

In [8]:
import numpy as np
import cv2
from keras.models import load_model

BASEPATH = './'
MODELPATH = './diego.h5'

#Prueba en tiempo real del modelo del modelo

emotion_dict = {0: "Angry", 1: "Happy", 2: "Sad", 3: "Surprise", 4: "Neutral"}

model = load_model('./diego.h5')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        prediction = model.predict(cropped_img)
        cv2.putText(frame, emotion_dict[int(np.argmax(prediction))], 
        (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()